In [1]:
import torch
import numpy as np
import cv2
import glob
import os

device = torch.device('cuda')

In [2]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = torch.nn.Sequential(
        
        torch.nn.Conv2d(3,64, kernel_size=3, padding=2),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2),
            
        torch.nn.Conv2d(64,64, kernel_size=5, padding=2),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2),
        )
        
        self.cls = torch.nn.Sequential(
        torch.nn.Linear(64 * 7 *7, 128),
        torch.nn.ReLU(),
        torch.nn.Linear(128,10)
        )
        
    def forward(self, x):
        out = self.conv(x)
        out = out.reshape(-1,64*7*7)
        out = self.cls(out)
        return out

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, path):
        super().__init__()
        self.image_paths = glob.glob(path)
        self.n_samples = len(self.image_paths)
    
    def __getitem__(self, sample_n):
        img_path = self.image_paths[sample_n]
        img = cv2.imread(img_path)
        label = int(os.path.basename(img_path).split("_")[0])
        return img, label
        
        
    def __len__(self):
        return self.n_samples

In [7]:
train_ds = Dataset("./mnist/train/*")
# train_dl = torch.utils.data.dataloader(train_ds, batch_size=32)
train_dl = torch.utils.data.DataLoader(
    train_ds,
    batch_size=128,
    shuffle=True,
    num_workers=4
)

In [10]:
val_ds = Dataset("./mnist/val/*")
# train_dl = torch.utils.data.dataloader(train_ds, batch_size=32)
val_dl = torch.utils.data.DataLoader(
    val_ds,
    batch_size=128,
    shuffle=True,
    num_workers=4
)

In [8]:
model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters())
lossf = torch.nn.CrossEntropyLoss()

In [ ]:
epochs = 10

for epoch in range(epochs):
    for img, label in train_dl:
        
        optimizer.zero_grad()     
        
        img = img.float().to(device).permute(0,3,1,2)
        label = label.to(device)
        
        output = model(img)
        
        loss = lossf(output, label)
        loss.backward()
        
        optimizer.step()
    accs = []
    for batch in val_dl:

        img, label = batch
        img = img.float().to(device).permute(0,3,1,2)
        label = label.to(device)
        with torch.no_grad():
            output = model(img)

        predict = output.argmax(1)
        acc = (predict == label).float().mean().detach().cpu().numpy()
        accs.append(acc)
    print(np.mean(accs))

0.9783203


Exception ignored in: <function _releaseLock at 0x7f1ec0efb5e0>
Traceback (most recent call last):
  File "/home/tudor/.miniconda3/envs/dlu/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "/home/tudor/.miniconda3/envs/dlu/lib/python3.9/site-packages/ipykernel/iostream.py", line 466, in flush
    if not evt.wait(self.flush_timeout):
  File "/home/tudor/.miniconda3/envs/dlu/lib/python3.9/threading.py", line 574, in wait
    KeyboardInterruptERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

Exception ignored in: <function _releaseLock at 0x7f1ec0efb5e0>
Traceback (most recent call last):
  File "/home/tudor/.miniconda3/envs/dlu/lib/python3.9/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
